# Guided Capstone 2 - NBA Win Model Prediction Using Machine Learning

# Section 0: Importing Libraries 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math
from datetime import datetime

# Section 1: Data Import and combination

In our investigation, our attempt to recreate the prior investigators model will require data of the same time period and features. In the prior investigation, the data used produce their NBA win prediction model covered a 12 season span, between the seasons of 2008 - 2009 to 2019 - 2020. 

Kaggale offers several databases that include the data portion mentioned above, and can be wrangled to reflect the data of the prior investigation 

# Section 1 Data Wrangling

# 1.1 Data Stitching

From Kaggle, separate sets have been acquired for both individual players statistics per game, and team statistics per game. These sets are separated by season.

In [2]:
#Below code appends datasets together based on file names.
start_year = 2008
end_year = 2009

team_stats = pd.DataFrame()
player_stats = pd.DataFrame()

for i in range(12) :
    player_stats_file_path = '/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/player_stats/' + str(start_year) + "-" + str(end_year) + '.csv'
    team_stats_file_path = '/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/team_stats/' + str(start_year) + "-" + str(end_year) + '.csv'
    player_stats = player_stats.append(pd.read_csv(player_stats_file_path))
    team_stats = team_stats.append(pd.read_csv(team_stats_file_path))
    
    start_year = start_year + 1
    end_year = end_year + 1

In [3]:
player_stats.shape

(373069, 26)

In [4]:
player_stats.head()

,Player,Date,Team,Against,Home,MP,FG,FGA,FG%,3P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,GameLink
0,LeBron James,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,36.00,9.0,21.0,0.429,0.0,...,6.0,7.0,6.0,2.0,1.0,3.0,4.0,22.0,-3.0,https://www.basketball-reference.com/boxscores...
1,Delonte West,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,35.29,2.0,4.0,0.500,0.0,...,2.0,2.0,2.0,0.0,0.0,1.0,0.0,6.0,-10.0,https://www.basketball-reference.com/boxscores...
2,Mo Williams,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,31.18,4.0,10.0,0.400,3.0,...,3.0,3.0,2.0,1.0,0.0,4.0,5.0,12.0,3.0,https://www.basketball-reference.com/boxscores...
3,Zydrunas Ilgauskas,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,30.24,3.0,5.0,0.600,0.0,...,5.0,8.0,1.0,0.0,0.0,4.0,3.0,15.0,7.0,https://www.basketball-reference.com/boxscores...
4,Ben Wallace,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,19.03,1.0,1.0,1.000,0.0,...,4.0,4.0,0.0,0.0,1.0,1.0,2.0,2.0,-1.0,https://www.basketball-reference.com/boxscores...


In [5]:
player_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 373069 entries, 0 to 27739
Data columns (total 26 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Player    373069 non-null  object 
 1   Date      373069 non-null  object 
 2   Team      373069 non-null  object 
 3   Against   373069 non-null  object 
 4   Home      373069 non-null  int64  
 5   MP        311326 non-null  float64
 6   FG        311326 non-null  float64
 7   FGA       311326 non-null  float64
 8   FG%       295932 non-null  float64
 9   3P        311326 non-null  float64
 10  3PA       311326 non-null  float64
 11  3P%       199196 non-null  float64
 12  FT        311326 non-null  float64
 13  FTA       311326 non-null  float64
 14  FT%       179762 non-null  float64
 15  ORB       311326 non-null  float64
 16  DRB       311326 non-null  float64
 17  TRB       311326 non-null  float64
 18  AST       311326 non-null  float64
 19  STL       311326 non-null  float64
 20  BLK  

In [6]:
team_stats.shape

(15348, 124)

In [7]:
team_stats.head()

,gmDate,seasonType,season,teamWins,teamLosses,teamAbbr,teamLoc,teamRslt,teamDayOff,teamPTS,...,opptOrtg,opptDrtg,opptEDiff,opptPlay%,opptAR,opptAST/TO,opptSTL/TO,opptPoss,opptPace,matchWinner
0,2008-10-28,Regular,2008-09,0,1,CLE,Away,Loss,0.0,85.0,...,96.9,91.5,5.4,0.41,17.57,1.00,0.62,92.9,92.9,BOS
1,2008-10-28,Regular,2008-09,0,1,MIL,Away,Loss,0.0,95.0,...,120.3,105.8,14.5,0.48,28.53,1.50,0.50,89.8,89.8,CHI
2,2008-10-28,Regular,2008-09,0,1,POR,Away,Loss,0.0,76.0,...,107.7,85.3,22.4,0.44,18.08,1.27,0.53,89.1,89.1,LAL
3,2008-10-29,Regular,2008-09,1,0,TOR,Away,Win,0.0,95.0,...,99.8,112.8,-13.0,0.37,14.47,0.88,0.18,84.2,84.2,TOR
4,2008-10-29,Regular,2008-09,1,0,NJN,Away,Win,0.0,95.0,...,94.6,105.7,-11.1,0.36,17.86,1.20,0.27,89.9,89.9,NJN


In [8]:
team_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15348 entries, 0 to 1142
Columns: 124 entries, gmDate to matchWinner
dtypes: float64(110), int64(4), object(10)
memory usage: 14.6+ MB


In [9]:
team_stats.to_csv('Team_Stats.csv')

# 1.2 Cleaning player_stats

In [10]:
#Reorder columns and naming
col = ['Player', 'Team', 'Against', 'Date']
other = [x for x in player_stats.columns if x not in col]
player_stats= player_stats[col + other]

In [11]:
#Dropping unecessary columns

player_stats.drop(columns = ['FG%', '3P%', 'FT%', 'GameLink'], inplace = True)

#'GameLink' turns out to be strings with no interesting information. If the full link was available, possible use to create Game IDs based on URL, however, that is not the case, and this column should be removed as it provides nothing of value.

In [12]:
#Checking for isnull()

player_stats.isna().sum()

Player         0
Team           0
Against        0
Date           0
Home           0
MP         61743
FG         61743
FGA        61743
3P         61743
3PA        61743
FT         61743
FTA        61743
ORB        61743
DRB        61743
TRB        61743
AST        61743
STL        61743
BLK        61743
TOV        61743
PF         61743
PTS        61743
+/-        61793
dtype: int64

While evaluating the NA values, we see that we do have a good chunk of NaN data. It is clear that we are not looking at players who received 0 stats sucha as 0 pts or rebounds. Instead the data clearly records players who played zero mins have NaN's in their stats. 

In the +/- column, there's always a chance that the player contributed, but their +/- is still 0. This is quite possible as +/- is a stat recorded based on both team performances during the time the player is playing. 

In [13]:
player_stats = player_stats[player_stats['MP'].notna()]

In [14]:
player_stats.isna().sum()

Player      0
Team        0
Against     0
Date        0
Home        0
MP          0
FG          0
FGA         0
3P          0
3PA         0
FT          0
FTA         0
ORB         0
DRB         0
TRB         0
AST         0
STL         0
BLK         0
TOV         0
PF          0
PTS         0
+/-        50
dtype: int64

In [15]:
#Converting 'Date' column to datetime object

player_stats['Date'] = pd.to_datetime(player_stats['Date'])

In [16]:
player_stats[player_stats['Home'] == 1].head()

,Player,Team,Against,Date,Home,MP,FG,FGA,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
12,Ray Allen,Boston Celtics,Cleveland Cavaliers,2008-10-28,1,40.39,2.0,9.0,1.0,4.0,...,1.0,3.0,4.0,1.0,1.0,0.0,4.0,1.0,8.0,-1.0
13,Paul Pierce,Boston Celtics,Cleveland Cavaliers,2008-10-28,1,38.13,10.0,19.0,2.0,4.0,...,1.0,2.0,3.0,4.0,1.0,0.0,2.0,4.0,27.0,12.0
14,Kevin Garnett,Boston Celtics,Cleveland Cavaliers,2008-10-28,1,36.09,5.0,15.0,0.0,0.0,...,1.0,5.0,6.0,2.0,2.0,0.0,2.0,1.0,11.0,5.0
15,Rajon Rondo,Boston Celtics,Cleveland Cavaliers,2008-10-28,1,29.57,4.0,5.0,0.0,0.0,...,1.0,4.0,5.0,6.0,3.0,0.0,2.0,5.0,14.0,4.0
16,Kendrick Perkins,Boston Celtics,Cleveland Cavaliers,2008-10-28,1,20.50,1.0,2.0,0.0,0.0,...,4.0,4.0,8.0,0.0,0.0,0.0,2.0,6.0,2.0,-7.0


In [17]:
test1 = player_stats[player_stats['Date'] == '2020-8-14']
test1

,Player,Team,Against,Date,Home,MP,FG,FGA,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
25566,Jerami Grant,Denver Nuggets,Toronto Raptors,2020-08-14,0,18.18,1.0,6.0,0.0,3.0,...,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0
25567,Paul Millsap,Denver Nuggets,Toronto Raptors,2020-08-14,0,15.30,4.0,8.0,2.0,3.0,...,1.0,2.0,3.0,2.0,2.0,1.0,1.0,2.0,10.0,-4.0
25568,Torrey Craig,Denver Nuggets,Toronto Raptors,2020-08-14,0,14.58,1.0,6.0,1.0,3.0,...,1.0,3.0,4.0,3.0,0.0,0.0,1.0,0.0,3.0,-6.0
25569,Jamal Murray,Denver Nuggets,Toronto Raptors,2020-08-14,0,10.12,4.0,5.0,3.0,3.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,11.0,-6.0
25570,Nikola Jokić,Denver Nuggets,Toronto Raptors,2020-08-14,0,10.12,1.0,1.0,0.0,0.0,...,0.0,1.0,1.0,2.0,0.0,0.0,1.0,0.0,2.0,-6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25648,Chris Clemons,Houston Rockets,Philadelphia 76ers,2020-08-14,1,16.36,2.0,7.0,0.0,5.0,...,0.0,1.0,1.0,4.0,1.0,1.0,1.0,4.0,4.0,-23.0
25649,Michael Frazier,Houston Rockets,Philadelphia 76ers,2020-08-14,1,16.36,0.0,4.0,0.0,4.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,-23.0
25650,Luc Mbah a Moute,Houston Rockets,Philadelphia 76ers,2020-08-14,1,13.55,1.0,3.0,0.0,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,-14.0
25651,Austin Rivers,Houston Rockets,Philadelphia 76ers,2020-08-14,1,12.21,0.0,2.0,0.0,2.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,-16.0


In [18]:
player_stats['Team'].unique()

array(['Cleveland Cavaliers', 'Boston Celtics', 'Milwaukee Bucks',
       'Chicago Bulls', 'Portland Trail Blazers', 'Los Angeles Lakers',
       'Toronto Raptors', 'Philadelphia 76ers', 'New Jersey Nets',
       'Washington Wizards', 'Atlanta Hawks', 'Orlando Magic',
       'Miami Heat', 'New York Knicks', 'Phoenix Suns',
       'San Antonio Spurs', 'Oklahoma City Thunder', 'Sacramento Kings',
       'Minnesota Timberwolves', 'Indiana Pacers', 'Detroit Pistons',
       'Memphis Grizzlies', 'Houston Rockets', 'Denver Nuggets',
       'Utah Jazz', 'Los Angeles Clippers', 'New Orleans Hornets',
       'Golden State Warriors', 'Charlotte Bobc', 'Dallas Mavericks',
       'Charlotte Bobcats', 'Brooklyn Nets', 'New Orleans Pelicans',
       'Charlotte Hornets'], dtype=object)

In [19]:
player_stats.Home.isnull().sum()

0

In [20]:
#Above we see that there are teams that have moved and the names have changed, since our data requires their prior season performances
#we will try to set the GameID's to have consistency with the team and their history (per the duration of our dataset)
#NJN are now the BKN 
name_change = player_stats[['Team', 'Against', 'Home']]



name_change['Team'] = name_change['Team'].replace({'New York Knicks' : 'NYK', 'New Jersey Nets': 'BKN', 'Brooklyn Nets':'BKN', 'New Orleans Hornets':'NOP', 'Charlotte Bobcats':'CHO', 'Charlotte Bobc':'CHO', 'Charlotte Hornets':'CHO', 'New Orleans Pelicans':'NOP', 'Los Angeles Lakers':'LAL', 'Los Angeles Clippers':'LAC' })
name_change['Against'] = name_change['Against'].replace({'New York Knicks' : 'NYK', 'New Jersey Nets': 'BKN', 'Brooklyn Nets':'BKN', 'New Orleans Hornets':'NOP', 'Charlotte Bobcats':'CHO', 'Charlotte Bobc':'CHO', 'Charlotte Hornets':'CHO', 'New Orleans Pelicans':'NOP', 'Los Angeles Lakers':'LAL', 'Los Angeles Clippers':'LAC' })
#New Orleans Hornets are now the New Orlean Pelicans
#Charlotte Bobcats are the charlotte hornets 
name_change['Team'].unique()


<ipython-input-20-db0d992679ce>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_change['Team'] = name_change['Team'].replace({'New York Knicks' : 'NYK', 'New Jersey Nets': 'BKN', 'Brooklyn Nets':'BKN', 'New Orleans Hornets':'NOP', 'Charlotte Bobcats':'CHO', 'Charlotte Bobc':'CHO', 'Charlotte Hornets':'CHO', 'New Orleans Pelicans':'NOP', 'Los Angeles Lakers':'LAL', 'Los Angeles Clippers':'LAC' })
<ipython-input-20-db0d992679ce>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_change['Against'] = n

array(['Cleveland Cavaliers', 'Boston Celtics', 'Milwaukee Bucks',
       'Chicago Bulls', 'Portland Trail Blazers', 'LAL',
       'Toronto Raptors', 'Philadelphia 76ers', 'BKN',
       'Washington Wizards', 'Atlanta Hawks', 'Orlando Magic',
       'Miami Heat', 'NYK', 'Phoenix Suns', 'San Antonio Spurs',
       'Oklahoma City Thunder', 'Sacramento Kings',
       'Minnesota Timberwolves', 'Indiana Pacers', 'Detroit Pistons',
       'Memphis Grizzlies', 'Houston Rockets', 'Denver Nuggets',
       'Utah Jazz', 'LAC', 'NOP', 'Golden State Warriors', 'CHO',
       'Dallas Mavericks'], dtype=object)

In [21]:
len(name_change['Against'].unique())
#30 teams in the NBA 

30

In [22]:
len(name_change['Team'].unique())

30

In [ ]:
for x in name_change['Home']:
    if x==1:
        player_stats['homeAbbr'] = name_change['Team'].str[0:3].str.upper()
    else:
        player_stats['homeAbbr'] = name_change['Against'].str[0:3].str.upper()

In [ ]:
player_stats['homeAbbr'].unique()

In [ ]:
player_stats['homeAbbr'] = player_stats['homeAbbr'].replace({'SAN':'SAS', 'OKL': 'OKC', 'GOL':'GSW'})
player_stats['homeAbbr'].unique()

In [ ]:
len(player_stats['homeAbbr'].unique())

In [ ]:
player_stats['gameID'] =  player_stats['Date'].dt.strftime('%Y%m%d') + player_stats['homeAbbr']
player_stats.head()

In [ ]:
player_stats.reset_index(drop = True, inplace = True)

In [ ]:
#Removing non-regular season games
dates =[['2008-10-28', '2009-04-16'],['2009-10-27', '2010-04-17'], ['2010-10-26', '2011-04-13' ], ['2011-12-25', '2012-04-26'], ['2012-10-30', '2013-04-17'], ['2013-10-29', '2014-04-16'], ['2014-10-28', '2015-04-15'], ['2015-10-27', '2016-04-16'], ['2016-10-25', '2017-04-12'], ['2017-10-17', '2018-04-11'], ['2018-10-16', '2019-04-10'], ['2019-10-22', '2020-08-14' ]]
is_regsea = lambda date : (date>= pd.to_datetime(start_date)) & (date<= pd.to_datetime(end_date))
all_regsea = pd.DataFrame()

for start_date, end_date in dates:
    print(start_date, end_date)
    all_regsea = pd.concat([all_regsea, player_stats.Date.apply(is_regsea)], axis=1)


In [ ]:
all_regsea.info()

In [ ]:
all_regsea = all_regsea.sum(axis=1)

In [ ]:
regsea = np.where(all_regsea == 1)[0]
regsea

In [ ]:
player_stats = player_stats.loc[regsea]

In [ ]:
player_stats.shape

In [ ]:
player_stats.drop(columns = ['homeAbbr'], inplace=True)

In [ ]:
player_stats.sort_values(by = 'Date', ascending = False, inplace = True)
player_stats.reset_index(inplace = True, drop = True)

In [ ]:
clean_player_stats = player_stats

In [ ]:
clean_player_stats.head()

# 1.3 Saving Cleaned Player Stats Data To CSV

In [ ]:
clean_player_stats.to_csv('Cleaned_Player_Stats.csv')

# 1.3 Cleaning team_stats

In [ ]:
#read in team_stats
team_stats = pd.read_csv('Team_Stats.csv')

In [ ]:
team_stats.reset_index(drop=True)

In [ ]:
#Converting 'gmDate' column to datetime object

team_stats['gmDate'] = pd.to_datetime(team_stats['gmDate'])

In [ ]:
team_stats= team_stats.iloc[:,1:]
team_stats['opptAbbr'].unique()

In [ ]:
#Our analysis will focus on regular season for the recreating portion of our anaylysis. 
team_stats['seasonType'].unique()

In [ ]:
#We will separate the data into regular season and playoffs.
team_stats = team_stats[team_stats['seasonType']=='Regular']

In [ ]:
team_stats['seasonType'].unique()

In [ ]:
#The games stats are recorded as team vs. opponent with another column indicating whether this is a home game or an away game. 
#This format leaves me to suspect there is double data of the same game. Let's investigate. 

team_stats['gmDate'] = pd.to_datetime(team_stats['gmDate'])
team_stats[['teamAbbr', 'teamLoc', 'opptAbbr', 'opptLoc', 'gmDate', 'opptLoc']][team_stats['gmDate'] == datetime(2020,8,14)]

#Seems our suspicion was wrong and away_games=team as home_games=opp. We will keep this in mind during feature engineering.
#Double check and seems Allstar games are not included as well. 

In [ ]:
team_stats[['teamAbbr', 'teamLoc', 'opptAbbr', 'opptLoc']][team_stats['gmDate'] == datetime(2020,8,14)]

In [ ]:
team_stats.reset_index(inplace = True, drop = True)

In [ ]:
all_regsea2 = pd.DataFrame()
is_regsea = lambda date : (date>= pd.to_datetime(start_date)) & (date<= pd.to_datetime(end_date))

for start_date, end_date in dates:
    print(start_date, end_date)
    all_regsea2 = pd.concat([all_regsea2, team_stats.gmDate.apply(is_regsea)], axis=1)

In [ ]:
all_regsea2 = all_regsea2.sum(axis=1)

In [ ]:
regsea2 = np.where(all_regsea2 == 1)[0]
regsea2

In [ ]:
team_stats = team_stats.loc[regsea2]

In [ ]:
team_stats.reset_index(inplace = True, drop = True)

In [ ]:
#The games stats are recorded as team vs. opponent with another column indicating whether this is a home game or an away game. 
#This format leaves me to suspect there is double data of the same game. Let's investigate. 

team_stats['gmDate'] = pd.to_datetime(team_stats['gmDate'])
team_stats[['teamAbbr', 'teamLoc', 'opptAbbr', 'opptLoc']][team_stats['gmDate'] == datetime(2020,8,14)]

#Seems our suspicion was wrong and away_games=team as home_games=opp. We will keep this in mind during feature engineering. 

In [ ]:
len(team_stats['opptAbbr'].unique())

In [ ]:
#33 teams, 3 more than what we had with the player stats, that means we have 3 team abbreviations that are duplicates
#CHA were the charlotte bobcats that because the new charlotto hornets CHO 
#NJN became the BKN
#NOH changed name to NOP
team_name_change = team_stats['opptAbbr'].replace({'NJN':'BKN', 'CHA':'CHO', 'BRK':'BKN', 'NOH' : 'NOP'})

In [ ]:
len(team_name_change.unique())

In [ ]:
dateString =  team_stats['gmDate'].dt.strftime('%Y%m%d')
dateString.head()

In [ ]:
team_stats['GameID'] = dateString + team_name_change
team_stats['GameID'].head()

In [ ]:
team_stats[['GameID','teamAbbr', 'opptAbbr', 'opptLoc']][team_stats['gmDate'] == datetime(2020,8,14)]

In [ ]:
#Renaming Columns

team_stats.rename(columns = {'gmDate' : 'Date', 'teamAbbr' : 'A_Team', 'opptAbbr' : 'H_Team' ,'teamFGM': 'teamFGMade', 'team2PM' : 'team2PMade' , 'team3PM' : 'team3PMade', 'teamFTM' : 'teamFTMade',   'opptFGM': 'opptFGMade', 'oppt2PM': 'oppt2PMade', 'oppt3PM' : 'oppt3PMade', 'opptFTM' : 'opptFTMade'}, inplace = True)

In [ ]:
#Removing percentage columns. 

team_stats.drop(columns = ['seasonType', 'teamFG%', 'team2P%', 'team3P%', 'teamFT%', 'teamTREB%', 'teamASST%', 'teamTS%', 'teamEFG%', 'teamOREB%', 'teamLoc', 'teamDREB%', 'teamTO%', 'teamSTL%', 'teamBLK%', 'teamBLKR', 'teamPPS', 'teamFIC', 'teamFIC40' , 'teamPlay%', 'teamAR', 'teamPoss', 'teamPace', 'teamSTL/TO', 'teamEDiff', 'opptEDiff', 'teamAST/TO', 'opptAST/TO', 'opptFG%', 'oppt2P%', 'oppt3P%', 'opptFT%', 'opptTREB%', 'opptASST%', 'opptTS%', 'opptEFG%', 'opptLoc', 'opptOREB%', 'opptDREB%', 'opptTO%', 'opptSTL%', 'opptBLK%', 'opptBLKR', 'opptPPS', 'opptFIC', 'opptFIC40', 'opptPlay%', 'opptAR', 'opptSTL/TO', 'opptPoss', 'opptPace'], inplace = True)

In [ ]:
team_stats['H_Loc'] = team_stats['H_Team']

In [ ]:
#Creating Missed Columns

team_stats['teamFGMissed'] = team_stats['teamFGA'] - team_stats['teamFGMade']
team_stats['team2PMissed'] = team_stats['team2PA'] - team_stats['team2PMade'] 
team_stats['team3PMissed'] = team_stats['team3PA'] - team_stats['team3PMade']
team_stats['opptFGMissed'] = team_stats['opptFGA'] - team_stats['opptFGMade']
team_stats['oppt2PMissed'] = team_stats['oppt2PA'] - team_stats['oppt2PMade']
team_stats['oppt3PMissed'] = team_stats['oppt3PA'] - team_stats['oppt3PMade']

In [ ]:
team_stats.sort_values(by = 'Date', ascending = False, inplace=True)
team_stats.reset_index(inplace = True, drop = True)

In [ ]:
team_stats.columns = [col.replace('team', 'A_') for col in team_stats.columns]
team_stats.columns = [col.replace('oppt', 'H_') for col in team_stats.columns]

In [ ]:
team_stats.columns

In [ ]:
#Reorder the columns for easier visuals

def alternate_lists(l_1, l_2) :
    l = []
    for i in range(len(l_1)) :
        l.append(l_1[i])
        l.append(l_2[i])
    return l

new_column_order = []
l = ['Date', 'GameID', 'season', 'H_Team', 'H_Wins', 'H_Losses', 'A_Team','A_Wins', 'A_Losses', 'H_Loc']

other = [x for x in team_stats.columns if x not in l] 

for i in range (1, 10) :
    h_stats_for_given_quarter = [x for x in other if x[-1] == str(i) and 'H_' in x]
    a_stats_for_given_quarter = [x for x in other if x[-1] == str(i) and 'A_' in x]
    
    overall_stats = alternate_lists(h_stats_for_given_quarter, a_stats_for_given_quarter)
    new_column_order += overall_stats

h_full_game_stats = [x for x in other if x not in new_column_order and 'H_' in x]
a_full_game_stats = [x for x in other if x not in new_column_order and 'A_' in x]
overall_full_game_stats = alternate_lists(h_full_game_stats, a_full_game_stats)

new_column_order = l + overall_full_game_stats + new_column_order

cleaned_team_stats= team_stats[new_column_order]

In [ ]:
cleaned_team_stats

In [ ]:
len(overall_full_game_stats)

In [ ]:
len(new_column_order)

In [ ]:
cleaned_team_stats.to_csv('cleaned_team_stats')

# Section 2 Feature Engineering

Our main goal is to predict winning and losing of games between teams. The stats teams and players accrue in games are not as helpful, therefore, we will engineer a few features that will provide more insight into win/loss predictions. The prior investigation utilized the following. 

1. Elo Ratings 
The Elo system is a calculate stat that keeps track of relative skill and quality of teams within the league. Everyteam starts with the same Elo rating at the beggining of a season, and will either add to or subtract value as they win/lose throughoput the season. 

2. Recent Team Performances 
We will keep track of the rolling averages for a teams performance over the last 10 games prior to the game we are predicting. This gives us a glimps of how the team is playing recently leading up to the predicted game. 

3. Recent Player Performances 
Keeping track of player performances can also tell us how well that player will play (suspicious of paisson model behaviour for players and their scoring in view of their seasonal stats) 

4. Player Performance In Season
Likewise, how this player has performed up to their current game within the season can also gives us glimpse into how their performances are trending. 

# 2.1 Elo Ratings

## Read in Team stats

In [ ]:
team_stats = pd.read_csv('cleaned_team_stats')
team_stats['Date'] = pd.to_datetime(team_stats['Date'])
team_stats.drop(columns = team_stats.columns[0], axis=1, inplace = True)
team_stats.info()


Elo rating takes into account variables suc as difficult of schedule, home court advantage, margin of victory, and etc. These variables give more insight than simply utilizing team win record percentages. The formula below was concepted by [Nate Silver](https://fivethirtyeight.com/features/how-we-calculate-nba-elo-ratings/#:~:text=Here's%20the%20formula%3A%20Take%20the,and%20then%20divide%20by%2028.). 

All teams start off with a 1500 elo rating. After each game, the winner is award addition elo value and the loser has their elo value subtracted. The equation to calculate for Elo is shown below, where $R_{i}$ is the current Elo rating.

$$ R_{i + 1} = k * (S_{team} - E_{team} + R_{i}) $$


$S_{team} = 1 $ when the team wins, and 0 if the team loses. The $E_{team}$ value is expected win probability of the team which is further defined as 

$$ E_{team} = \frac{1}{1+10^{\frac{opp\_elo - team\_elo}{400}}}$$

The $k$ in the the Elo equationis a moving constant depending on the margin of victory and difference in Elo ratings. The k constant is calculated as: 

$$ k = 20 \frac{(MOV_{winner} + 3)^{0.8}}{7.5 + 0.006(elo\_difference_{winner})} $$

Elo ratings carry over from season to season, as teams who were good previous season, usually do not have significant drop offs from previous seasons. If $R$ is a teams final Elo rating for a season, than below is the Elo rating to being their next season. 

$$ (R * 0.75) + (0.25 * 0501)$$

## Def Elo Variable Functions

Function for win probabilty as per Nate Silver [Win Prob](https://fivethirtyeight.com/features/introducing-nfl-elo-ratings/)

$$ Pr(Home) = \frac{1}{10^{(-EloDiff/400)}+1}$$

In [ ]:
#Define function to calculate Win Probability implied by Elo ratings and home court adjustments
def win_probs(home_elo, away_elo, home_court_advantage) :
    h = math.pow(10, home_elo/400)
    r = math.pow(10, away_elo/400)
    a = math.pow(10, home_court_advantage/400)
    
    denom = r + a*h
    home_prob = a*h / denom
    away_prob = r / denom
    
    return home_prob, away_prob

#odds the home team will win based on elo ratings and home court advantage

def home_odds_on(home_elo, away_elo, home_court_advantage) :
    h = math.pow(10, home_elo/400)
    r = math.pow(10, away_elo/400)
    a = math.pow(10, home_court_advantage/400)
    
    return a*h/r

#this function determines the constant used in the elo rating, based on margin of victory and difference in elo ratings
def elo_k(MOV, elo_diff):
    k = 20
    if MOV>0:
        multiplier=(MOV+3)**(0.8)/(7.5+0.006*(elo_diff))
    else:
        multiplier=(-MOV+3)**(0.8)/(7.5+0.006*(-elo_diff))
    return k*multiplier

#updates the home and away teams elo ratings after a game 

def update_elo(home_score, away_score, home_elo, away_elo, home_court_advantage) :
    home_prob, away_prob = win_probs(home_elo, away_elo, home_court_advantage) 
    
    if (home_score - away_score > 0) :
        
        home_win = 1 
        away_win = 0 
    
    else :
        
        home_win = 0 
        away_win = 1 
        
    k = elo_k(home_score - away_score, home_elo - away_elo)
    
    updated_home_elo = home_elo + k * (home_win - home_prob) 
    updated_away_elo = away_elo + k * (away_win - away_prob)
    
    return updated_home_elo, updated_away_elo


#takes into account prev season elo
def get_prev_elo(team, date, season, team_stats, elo_df) :
    prev_game = team_stats[team_stats['Date'] < game_date][(team_stats['H_Team'] == team) | (team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(1).iloc[0] 
    
    if team == prev_game['H_Team'] :
        elo_rating = elo_df[elo_df['GameID'] == prev_game['GameID']]['H_Team_Elo_After'].values[0]
        
    else :
        elo_rating = elo_df[elo_df['GameID'] == prev_game['GameID']]['A_Team_Elo_After'].values[0]
        
    if prev_game['season'] != season :
        return (0.75 * elo_rating) + (0.25 * 1505)
    
    else :
        return elo_rating


## Calculating Elo Ratings

In [ ]:
team_stats.sort_values(by = 'Date', inplace = True)
team_stats.reset_index(inplace=True, drop = True)

elo_df = pd.DataFrame(columns=['GameID', 'H_Team', 'A_Team', 'H_Team_Elo_Before', 'A_Team_Elo_Before', 'H_Team_Elo_After', 'A_Team_Elo_After'])
teams_elo_df = pd.DataFrame(columns=['GameID','Team', 'Elo', 'Date', 'Where_Played', 'season']) 

for index, row in team_stats.iterrows(): 
    game_id = row['GameID']
    game_date = row['Date']
    season = row['season']
    h_team, a_team = row['H_Team'], row['A_Team']
    h_score, a_score = row['H_PTS'], row['A_PTS'] 
    
    if (h_team not in elo_df['H_Team'].values and h_team not in elo_df['A_Team'].values) :
        h_team_elo_before = 1500
    else :
        h_team_elo_before = get_prev_elo(h_team, game_date, season, team_stats, elo_df)
    
    if (a_team not in elo_df['H_Team'].values and a_team not in elo_df['A_Team'].values) : 
        a_team_elo_before = 1500
    
    else :
        a_team_elo_before = get_prev_elo(a_team, game_date, season, team_stats, elo_df)
    
    h_team_elo_after, a_team_elo_after = update_elo(h_score, a_score, h_team_elo_before, a_team_elo_before, 69)
    
    new_row = {'GameID': game_id, 'H_Team': h_team, 'A_Team': a_team, 'H_Team_Elo_Before': h_team_elo_before, 'A_Team_Elo_Before': a_team_elo_before, \
                                                                        'H_Team_Elo_After' : h_team_elo_after, 'A_Team_Elo_After': a_team_elo_after}
    teams_row_one = {'GameID': game_id,'Team': h_team, 'Elo': h_team_elo_before, 'Date': game_date, 'Where_Played': 'Home', 'season': season}
    teams_row_two = {'GameID': game_id,'Team': a_team, 'Elo': a_team_elo_before, 'Date': game_date, 'Where_Played': 'Away', 'season': season}

    elo_df = elo_df.append(new_row, ignore_index = True)
    teams_elo_df = teams_elo_df.append(teams_row_one, ignore_index=True)
    teams_elo_df = teams_elo_df.append(teams_row_two, ignore_index=True)
    
    
    

In [ ]:
#teams_elo_df.set_index(["Team"], append=True)
#dataset = teams_elo_df.pivot(index="Team",values="Elo", columns="Date")
dates = list(set([d.strftime("%m-%d-%Y") for d in teams_elo_df["Date"]]))
dates = sorted(dates, key=lambda x: datetime.strptime(x, '%m-%d-%Y'))
teams = team_stats["A_Team"]
dataset = pd.DataFrame(columns=dates)
dataset["Team"] = teams.drop_duplicates()
dataset = dataset.set_index("Team")
for index, row in teams_elo_df.iterrows():
    date = row["Date"].strftime("%m-%d-%Y")
    team = row["Team"]
    elo = row["Elo"]
    dataset[date][team] = elo

teams_elo_df['Elo'] = teams_elo_df['Elo'].astype(float)

elo_df

Reviewing the data in a visual medium

In [ ]:
%matplotlib inline
import random

def runningMeanFast(x, N):
    return np.convolve(x, np.ones((N,))/N)[(N-1):]
'''
for index, row in dataset.sample(frac=0.2).iterrows():
  sns.lineplot(data=row, x=dates, y=row.values)

plt.set(xlabel="Dates")
'''
elos = teams_elo_df['Elo'].values
#teams_elo_df['Elo_moving_average'] = runningMeanFast(elos, 15)
plt.figure(figsize=(20,10))
sample_teams = random.sample(list(teams), k=3)
sns.set(font_scale=1.5)
sampled_data = teams_elo_df.loc[teams_elo_df['Team'].isin(sample_teams)]
sns.lineplot(data=sampled_data, x='Date', y='Elo', hue='Team')
plt.title("ELO over Time")

## 2.2 Recent Team Performance 

### Teams avg stats in previous ten games

In [ ]:
team_stats.info()

In [ ]:
team_stats['GameID']

In [ ]:
team_stats.iloc[:, 14]

In [ ]:
team_stats.iloc[:, 53]

In [ ]:
def get_avg_stats(team, game_date, team_season_stats, n) :
    prior_games = team_season_stats[team_season_stats['Date'] < game_date][(team_season_stats['H_Team'] == team) | (team_season_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
    
    home_df = prior_games.iloc[:, range(3, 53 ,2)]
    home_df.columns = [x[2:] for x in home_df.columns]
    away_df = prior_games.iloc[:, range( 4, 54, 2)]
    away_df.columns = [x[2:] for x in away_df.columns ]
    
    df = pd.concat([home_df, away_df])
    df = df[df['Team']==team]
    df.drop(columns = ['Team'], inplace = True)
    
    return df.mean()

In [ ]:
#Recent Performance dataframe for team stats
rp_df = pd.DataFrame()


for Season in team_stats['season'].unique():
    c1 = ['Date', 'GameID', 'season', 'H_Team', 'A_Team']
    c2 = list(team_stats.columns[14:65])
    columns = c1 + c2
    
    team_season_stats = team_stats[cleaned_team_stats['season'] == Season ].sort_values(by='Date')[columns].reset_index(drop = True)
    
    season_recent_performance = pd. DataFrame()
    
    for index, row in team_season_stats.iterrows():
        game_ID = row['GameID']
        game_date = row['Date']
        h_team = row['H_Team']
        a_team = row['A_Team']
        
        h_team_recent_performance = get_avg_stats(h_team, game_date, team_season_stats, 10)
        h_team_recent_performance.index = ['H_Last_10_gm_avg_' + x for x in h_team_recent_performance.index]
        
        a_team_recent_performance = get_avg_stats(a_team, game_date, team_season_stats, 10)
        a_team_recent_performance.index = ['A_Last_10_gm_avg_' + x for x in a_team_recent_performance.index]
        
        nrow = pd.concat([h_team_recent_performance, a_team_recent_performance], sort = False)
        nrow['GameID'] = game_ID
        
        season_recent_performance = season_recent_performance.append(nrow, ignore_index = True)
        season_recent_performance = season_recent_performance[nrow.index]
    
    rp_df = pd.concat([rp_df, season_recent_performance])
    
    



In [ ]:
rp_df.dropna()

In [ ]:
#drop avg mins 

## 2.3 Combing average stats 

### Create a final_team_stats which combines data from the recent performance dataframe with the meta data of each game. 

In [ ]:
final_team_stats = team_stats.iloc[0 :, [0,1,2,3,6,14,15]].merge(elo_df.drop(columns=['H_Team', 'A_Team']), on = 'GameID') \
.merge(rp_df, on = 'GameID')

final_team_stats = final_team_stats.dropna()

In [ ]:
final_team_stats

In [ ]:
#Setting dataframes to combine into a final team performance df 
h_cols = final_team_stats.columns[final_team_stats.columns.str.startswith('H_')]

team_df = final_team_stats.iloc[0:, [0,1,2,3,4,9,10]].drop(columns = ['H_Team', 'A_Team'])
team_df_h = team_df.drop(columns=team_df.columns[team_df.columns.str.startswith('A_')])
team_df_a = team_df.drop(columns=team_df.columns[team_df.columns.str.startswith('H_')])

recent_perf_h = rp_df.drop(columns = rp_df.columns\
                           [rp_df.columns.str.startswith('A_')])
recent_perf_a = rp_df.drop(columns = rp_df.columns\
                           [rp_df.columns.str.startswith('H_')])

#Combining team_df, teams_elo_df, and recent_perf to create a team by team row df. 
tbt_h = team_df_h.merge(teams_elo_df[teams_elo_df.Where_Played == 'Home'], on = 'GameID') \
.merge(recent_perf_h, on ='GameID')

tbt_a = team_df_a.merge(teams_elo_df[teams_elo_df.Where_Played == 'Away'], on = 'GameID') \
.merge(recent_perf_h, on ='GameID')

#Removing 'H_' and 'A_', redundant information in our new df. _x, and _y will appear after first operator and will need to be removed

tbt_h.columns = tbt_h.columns.str.replace('H_', '')
tbt_h.columns = tbt_h.columns.str.replace('_x', '')
tbt_h.columns = tbt_h.columns.str.replace('_y', '')
tbt_a.columns = tbt_a.columns.str.replace('A_', '')
tbt_a.columns = tbt_a.columns.str.replace('_x', '')
tbt_a.columns = tbt_a.columns.str.replace('_y', '')
#Removing any duplicate columns 
tbt_h = tbt_h.loc[:, ~tbt_h.columns.duplicated()]
tbt_a = tbt_a.loc[:, ~tbt_a.columns.duplicated()]

#concating the away and home team by team dataframes by index
final_team_performances_df = pd.concat([tbt_h, tbt_a]).sort_index(axis=0).reset_index().drop(columns=['index'])

In [ ]:
final_team_performances_df

## 2.4 Label column and save df to csv

Creating a final "Label" that differentiates between winner and loser for our evaluation. 

In [ ]:
final_team_stats['Label'] = [1 if x > 0 else 0 for x in final_team_stats['H_PTS'] - final_team_stats['A_PTS']]

final_team_stats.drop(columns = ['H_PTS', 'A_PTS'], inplace = True)
final_team_stats = final_team_stats.dropna()
final_team_stats.reset_index(inplace = True)

combined_df = final_team_stats

final_team_stats

In [ ]:
final_team_stats.to_csv('final_team_stats.csv')

## 2.5 Creating Recent Player Performance DataFrame

Like the average team stats per 10 games, we will do the same for the player stats for their previous 10 games

### Read In Player Stats

In [ ]:
playerstats = pd.read_csv('Cleaned_Player_Stats.csv', index_col = 0)

playerstats = playerstats.sort_values(by='Date').reset_index(drop = True)

IDseason = final_team_stats[['season', 'GameID']].drop_duplicates()

playerstats.rename(columns={'gameID': 'GameID'}, inplace=True)

IDseason.head()

In [ ]:
playerstats.head()

In [ ]:
merge = playerstats.merge(IDseason , how='left', on='GameID')

In [ ]:
merge

## Define Function For Avg Stats 

In [ ]:
def rolling_average_last_n_games(n):
    playerstats_recent = pd.DataFrame
    
    for season in playerstats['season'].unique():
        season_stats = playerstats[playerstats['season']==season]
        
        for player in season_stats['Player'].unique():
            player_performance = season_stats[season_stats['Player']==player].rolling(n, min_periods=1).mean().shift(1)
            
            playerstats_recent = playerstats_recent.append(player_performance, ignore_index = False)
        
        #renaming columns to reflect 10 game avg
        playerstats_recent.columns = ['Last_' + str(n) + '_Avg_' + x for x in playerstats_recent.columns]
        #merging playerstats to playerstats_recent and at the same time removing the redundent current game avg stats.
        playerstats_recent = playerstats[['Player', 'GameID']].merge(playerstats_recent.drop(columns = 'Last_' + str(n) + '_Avg_' + 'GameID'), left_index=True, right_index=True)
        
        return playerstats_recent

In [ ]:
player_recent_performance = rolling_average_last_n_games(10)

player_recent_performance

# Previously version work to be sorted...

In [ ]:
#website - "https://www.kaggle.com/heeebsinc/nbaseasonstats201820"
#The above datasets are split into season from 2000 to 2020. We are only interested in 2008 - 2020.
cs_player= pd.read_csv('/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/player_stats/2008-2009.csv')

In [ ]:
cs_player.info()

In [ ]:
cs_player.head()

In [ ]:
print(cs_player['GameLink'])

In [ ]:
#Code for removing columns for future reference. 
#cs_player = cs_player.drop(['GameLink'], axis=1)

In [ ]:
#Question, when trying to add seasons to the dataset, it seems to immediately jump to 2019 instead of following
#the start year as noted below.
start_year = 2008
end_year = 2009

player_stats_combined_df = pd.DataFrame()

for i in range(12) :
  player_stats_file_path = '/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/player_stats/' + str(start_year) + "-" + str(end_year) + '.csv'

  player_stats_combined_df = player_stats_combined_df.append(pd.read_csv(player_stats_file_path))

  start_year = start_year + 1
  end_year = end_year + 1

In [ ]:
player_stats_combined_df.head()

In [ ]:
#set(player_stats_combined_df['Season'])

In [ ]:
player_stats_combined_df.info()

In [ ]:
player_stats_combined_df.describe()

In [ ]:
player_stats_combined_df.isnull().sum()

In [ ]:
player_stats_combined_df[player_stats_combined_df['3P%'].isnull()]

In [ ]:
threeptking = player_stats_combined_df[['Player','Team', 'Against', 'Date', '3PA', 'PTS']][player_stats_combined_df['3P%']==1]
threeptking[threeptking['3PA']>8]

In [ ]:
print(player_stats_combined_df[['Player','Date','3PA','Against','3P%']][player_stats_combined_df['3PA']>8])

In [ ]:
player_stats_combined_df['Ast/To'] = player_stats_combined_df['AST']/player_stats_combined_df['TOV']

In [ ]:
player_stats_combined_df = player_stats_combined_df.drop(['GameLink'], axis = 1)

In [ ]:
player_stats_combined_df['Date'] = pd.to_datetime(player_stats_combined_df['Date'])

In [ ]:
player_stats_combined_df['Date']

In [ ]:
#Dataset acquired from kaggle: 
#website - "https://www.kaggle.com/rafaelgreca/nba-games-box-score-since-1949"

cs_team = pd.read_csv('/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/team_stats/2019-2020.csv')

In [ ]:
cs_team.info()

In [ ]:
print(cs_team.columns.tolist())

In [ ]:
cs_team.head()

In [ ]:
start_year = 2008
end_year = 2009

team_stats_combined_df = pd.DataFrame()

for i in range(12) :
  team_stats_file_path = '/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/team_stats/' + str(start_year) + "-" + str(end_year) + '.csv'
  
  team_stats_combined_df = team_stats_combined_df.append(pd.read_csv(team_stats_file_path, index_col = 0))
  
  start_year = start_year + 1
  end_year = end_year + 1

In [ ]:
team_stats_combined_df.describe()

In [ ]:
team_stats_combined_df.isnull().sum()!=0

In [ ]:
team_stats_combined_df.columns[team_stats_combined_df.isnull().any()].tolist()

In [ ]:
#Dataset from prior investigation - NBA Combined Player Stats
#website - "https://towardsdatascience.com/predicting-the-outcome-of-nba-games-with-machine-learning-a810bb768f20"

ps_player = pd.read_csv('NBA_Combined_Player_Stats.csv', index_col= 0)

In [ ]:
ps_player.head()

In [ ]:
ps_player.info()

In [ ]:
## 2.2 REVIEWING PRIOR INVESTIGATION'S TEAM STATISTICS 

In [ ]:
#Dataset from prior investigation - NBA Combined Team Stats 
#website - "https://towardsdatascience.com/predicting-the-outcome-of-nba-games-with-machine-learning-a810bb768f20"

ps_team = pd.read_csv('/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/NBA_Combined_Team_Stats.csv', index_col=0)

In [ ]:
ps_team.info()

In [ ]:
ps_team.head()

In [ ]:
ps_team.columns.to_list()

In [ ]:
range(3, 43, 2)